In [1]:
import pandas as pd
from sklearn.preprocessing import normalize
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
file = Path('Resources/survey_results_1.xlsx')

In [3]:
results_df = pd.read_excel(file)
results_df.head()

,ID,Start time,Completion time,Email,Name,What is your age range?,Gender,Race/Ethnicity (Select all that apply),Which US Geographic Region do you live in?,What is the name of your favorite TV Show?\n,How often do you eat vegetables?,Types of veggies you most like to eat (pick your top 5!),"What is your ""go to"" choice for veggies at home? (Pick your top three)","What is your ""go to"" choice for veggies at a restaurant (top three)",Preferred ways for cooking/preparing the choices you selected above,What alternative to vegetables do you use/eat to stay healthy?,"What are your top three favorite types of fruits you like to eat? (example: apples, bananas, oranges)"
0,1,2022-06-04 15:01:02,2022-06-04 15:05:26,anonymous,NaN,40-49,Female,"Caucasian, non-Hispanic;",Midwest,Firefly,1/week,Asparagus ;Corn;Carrots;Spinach;Broccoli;,Lettuce (any kind);Carrots;Spinach;,Asparagus;Corn;Broccoli;,Grilled;Sauteed;Baked;,NaN,"Cherries, blueberries, pineapple"
1,2,2022-06-06 19:20:23,2022-06-06 19:22:18,anonymous,NaN,20-29,Female,Asian;,Midwest,Vincenzo,4-5/week,Broccoli;Cauliflower;Corn;Green Beans;Peas;Spi...,Broccoli;Corn;Peas;,Broccoli;Cauliflower;Corn;,Fried/Air-fried;Grilled;Sauteed;Microwaved;,NaN,"Mangoes, cherries, oranges"
2,3,2022-06-06 19:19:21,2022-06-06 19:22:26,anonymous,NaN,40-49,Female,"Caucasian, non-Hispanic;",Midwest,Ted Lasso,Daily 6-7 week,Broccoli;Carrots;Green Beans;Spinach;Squash;,Broccoli;Carrots;Squash;,Lettuce (any kind);Broccoli;Spinach;,Steamed;Baked;,NaN,"bananas, strawberries, mangos"
3,4,2022-06-06 19:20:24,2022-06-06 19:25:18,anonymous,NaN,20-29,Female,Black or African American;,Midwest,Succession,2-3/week,Broccoli;Carrots;Corn;Green Beans;Cauliflower;,Broccoli;Carrots;Cauliflower;,Broccoli;Carrots;Green Beans;,Baked;Fried/Air-fried;Steamed;,NaN,"Bananas, strawberries, oranges"
4,5,2022-06-06 19:28:42,2022-06-06 19:35:54,anonymous,NaN,30-39,Female,"Caucasian, non-Hispanic;",Midwest,Critical Role,2-3/week,Carrots;Spinach;Squash;Corn;Cauliflower;,Spinach;Carrots;Broccoli;,Spinach;Carrots;Squash;,Baked;Sauteed;,NaN,"Peaches, Oranges, Mangoes"


In [4]:
#RENAME COLUMNS
survey_df = results_df.rename(columns={
    "ID":"id",
    "Start time": "start_time",
    "Completion time": "end_time",
    "Email": "email",
    "Name": "name",	
    "What is your age range?": "age",
    "Gender":"gender",	
    "Race/Ethnicity (Select all that apply)": "race",	
    "Which US Geographic Region do you live in?": "region",	
    "What is the name of your favorite TV Show?\n": "tv_show",
    "How often do you eat vegetables?": "frequency",
    "Types of veggies you most like to eat (pick your top 5!)": "to_eat",
    'What is your "go to" choice for veggies at home? (Pick your top three)': "at_home",
    'What is your "go to" choice for veggies at a restaurant (top three)': "at_restaurant",	
    "Preferred ways for cooking/preparing the choices you selected above": "cook_prep",
    "What alternative to vegetables do you use/eat to stay healthy?": "alternatives",
    "What are your top three favorite types of fruits you like to eat? (example: apples, bananas, oranges)": "fruits"})

In [5]:
#SPLIT COLUMNS WITH MULTIPLE SELECTIONS

to_eat_test = survey_df["to_eat"].str.split(';', expand=True)
drop_list_five = [0,1,2,3,4]
drop_list_three = [0,1,2]

top_to_eat = to_eat_test.drop(to_eat_test.columns.difference(drop_list_five), axis=1)


top_to_eat = top_to_eat.rename(columns={0:"to_eat_1", 1: "to_eat_2", 2: "to_eat_3", 3: "to_eat_4", 4: "to_eat_5"})
top_to_eat
# top_five = to_eat_test[[0,1,2,3,4]]
# top_five


,to_eat_1,to_eat_2,to_eat_3,to_eat_4,to_eat_5
0,Asparagus,Corn,Carrots,Spinach,Broccoli
1,Broccoli,Cauliflower,Corn,Green Beans,Peas
2,Broccoli,Carrots,Green Beans,Spinach,Squash
3,Broccoli,Carrots,Corn,Green Beans,Cauliflower
4,Carrots,Spinach,Squash,Corn,Cauliflower
...,...,...,...,...,...
170,Corn,Broccoli,Cauliflower,Carrots,
171,Broccoli,Carrots,Green Beans,,None
172,Corn,Lettuce (any kind),Spinach,Carrots,
173,Carrots,Spinach,Broccoli,Cauliflower,Green Beans


In [6]:
# survey_df['to_eat_'{i+1}'] = top_to_eat[i]
# survey_df

In [7]:
survey_df["tv_show"] = survey_df["tv_show"].str.upper()
survey_df

,id,start_time,end_time,email,name,age,gender,race,region,tv_show,frequency,to_eat,at_home,at_restaurant,cook_prep,alternatives,fruits
0,1,2022-06-04 15:01:02,2022-06-04 15:05:26,anonymous,NaN,40-49,Female,"Caucasian, non-Hispanic;",Midwest,FIREFLY,1/week,Asparagus ;Corn;Carrots;Spinach;Broccoli;,Lettuce (any kind);Carrots;Spinach;,Asparagus;Corn;Broccoli;,Grilled;Sauteed;Baked;,NaN,"Cherries, blueberries, pineapple"
1,2,2022-06-06 19:20:23,2022-06-06 19:22:18,anonymous,NaN,20-29,Female,Asian;,Midwest,VINCENZO,4-5/week,Broccoli;Cauliflower;Corn;Green Beans;Peas;Spi...,Broccoli;Corn;Peas;,Broccoli;Cauliflower;Corn;,Fried/Air-fried;Grilled;Sauteed;Microwaved;,NaN,"Mangoes, cherries, oranges"
2,3,2022-06-06 19:19:21,2022-06-06 19:22:26,anonymous,NaN,40-49,Female,"Caucasian, non-Hispanic;",Midwest,TED LASSO,Daily 6-7 week,Broccoli;Carrots;Green Beans;Spinach;Squash;,Broccoli;Carrots;Squash;,Lettuce (any kind);Broccoli;Spinach;,Steamed;Baked;,NaN,"bananas, strawberries, mangos"
3,4,2022-06-06 19:20:24,2022-06-06 19:25:18,anonymous,NaN,20-29,Female,Black or African American;,Midwest,SUCCESSION,2-3/week,Broccoli;Carrots;Corn;Green Beans;Cauliflower;,Broccoli;Carrots;Cauliflower;,Broccoli;Carrots;Green Beans;,Baked;Fried/Air-fried;Steamed;,NaN,"Bananas, strawberries, oranges"
4,5,2022-06-06 19:28:42,2022-06-06 19:35:54,anonymous,NaN,30-39,Female,"Caucasian, non-Hispanic;",Midwest,CRITICAL ROLE,2-3/week,Carrots;Spinach;Squash;Corn;Cauliflower;,Spinach;Carrots;Broccoli;,Spinach;Carrots;Squash;,Baked;Sauteed;,NaN,"Peaches, Oranges, Mangoes"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,171,2022-06-08 07:22:36,2022-06-08 07:25:52,anonymous,NaN,50-65,Female,"Caucasian, non-Hispanic;",Midwest,CHOPPED,2-3/week,Corn;Broccoli;Cauliflower;Carrots;,Corn;Broccoli;,Corn;Broccoli;Lettuce (any kind);,Baked;,NaN,Blueberry
171,172,2022-06-08 08:45:06,2022-06-08 08:47:29,anonymous,NaN,40-49,Male,"Caucasian, Latino/Hispanic;",Midwest,SEINFELD,4-5/week,Broccoli;Carrots;Green Beans;,Broccoli;,Lettuce (any kind);,Microwaved;,NaN,"Bananas, strawberries, kumquat"
172,173,2022-06-08 08:47:55,2022-06-08 08:50:06,anonymous,NaN,40-49,Female,"Caucasian, non-Hispanic;",Midwest,DON'T HAVE ONE,Rarely (Only if someone makes me; holidays onl...,Corn;Lettuce (any kind);Spinach;Carrots;,Carrots;Corn;Lettuce (any kind);,Corn;,Steamed;,NaN,"Kiwi, mango, strawberries"
173,174,2022-06-08 21:35:14,2022-06-08 21:38:54,anonymous,NaN,50-65,Female,"Caucasian, non-Hispanic;",Midwest,JIMMY KIMMEL,4-5/week,Carrots;Spinach;Broccoli;Cauliflower;Green Beans;,Carrots;Spinach;Broccoli;,Broccoli;Cauliflower;Spinach;,Sauteed;,NaN,"Bananna's, apples, oranges"


In [8]:
drop_list_five = [0,1,2,3,4]
drop_list_three = [0,1,2]

# SPLIT COLUMNS WITH MULTIPLE SELECTIONS
survey1_df = to_eat_test = survey_df["to_eat"].str.split(';', expand = True)
at_home_test = survey_df["at_home"].str.split(';', expand = True)
at_restaurant_test = survey_df["at_restaurant"].str.split(';', expand = True)
cook_prep_test = survey_df["cook_prep"].str.split(';', expand = True)
fruits_test = survey_df["fruits"].str.split(',', expand = True)

# drop any columns that come after our drop list
top_to_eat = to_eat_test.drop(to_eat_test.columns.difference(drop_list_five), axis = 1)
top_at_home = at_home_test.drop(at_home_test.columns.difference(drop_list_three), axis = 1)
top_at_restaurant = at_restaurant_test.drop(at_restaurant_test.columns.difference(drop_list_three), axis = 1)
top_cook_prep = cook_prep_test.drop(cook_prep_test.columns.difference(drop_list_three), axis = 1)
top_fruits = fruits_test.drop(fruits_test.columns.difference(drop_list_three), axis = 1)

# rename the new columns
top_to_eat = top_to_eat.rename(columns = {0:"to_eat_1", 1: "to_eat_2", 2: "to_eat_3", 3: "to_eat_4", 4: "to_eat_5"})
top_at_home = top_at_home.rename(columns = {0:"at_home_1", 1: "at_home_2", 2: "at_home_3"})
top_at_restaurant = top_at_restaurant.rename(columns = {0:"at_restaurant_1", 1: "at_restaurant_2", 2: "at_restaurant_3"})
top_cook_prep = top_cook_prep.rename(columns = {0:"cook_prep_1", 1: "cook_prep_2", 2: "cook_prep_3"})
top_fruits = top_fruits.rename(columns = {0:"fruits_1", 1: "fruits_2", 2: "fruits_3"})

In [9]:
top_fruits

,fruits_1,fruits_2,fruits_3
0,Cherries,blueberries,pineapple
1,Mangoes,cherries,oranges
2,bananas,strawberries,mangos
3,Bananas,strawberries,oranges
4,Peaches,Oranges,Mangoes
...,...,...,...
170,Blueberry,None,None
171,Bananas,strawberries,kumquat
172,Kiwi,mango,strawberries
173,Bananna's,apples,oranges


In [10]:
#Creating list of new dataframes
df_list = [top_to_eat, top_at_home, top_at_restaurant,top_cook_prep, top_fruits]
# pd.merge(pd.merge(top_to_eat,top_at_home,on='id'),top_at_restaurant,on='id')

#Merging new dataframes into one
df = df_list[0]
for df_ in df_list[1:]:
    df = df.merge(df_, left_index=True, right_index=True)

#Adding columns back into survey_df 
for column in df:
     
    # Select column conents by column
    # name using [] operator
    survey_df[column] = df[column]

print(survey_df.keys())

Index(['id', 'start_time', 'end_time', 'email', 'name', 'age', 'gender',
       'race', 'region', 'tv_show', 'frequency', 'to_eat', 'at_home',
       'at_restaurant', 'cook_prep', 'alternatives', 'fruits', 'to_eat_1',
       'to_eat_2', 'to_eat_3', 'to_eat_4', 'to_eat_5', 'at_home_1',
       'at_home_2', 'at_home_3', 'at_restaurant_1', 'at_restaurant_2',
       'at_restaurant_3', 'cook_prep_1', 'cook_prep_2', 'cook_prep_3',
       'fruits_1', 'fruits_2', 'fruits_3'],
      dtype='object')


In [11]:
#Dependencies for API
import requests
import json

from api_key import movie_api

In [12]:
shows = survey_df["tv_show"].tolist()
print(shows)

['FIREFLY', 'VINCENZO', 'TED LASSO', 'SUCCESSION', 'CRITICAL ROLE', 'GOSSIP GIRL', 'DEMON SLAYER', 'WEST WING', 'FRIENDS', 'NCIS', 'NCIS', 'THE OFFICE', 'TOP CHEF', 'OUTLANDER', 'NEW GIRL', 'TWIN PEAKS', 'STRANGER THINGS', 'NONE', 'CHICAGO MED', 'NEW GIRL', 'BATTLEBOTS', 'BREAKING BAD', 'FAMILY FEUD ', 'CHICAGO MED, FIRE, PD', 'THE OFFICE', 'RICK AND MORTY', "HELL'S KITCHEN ", 'BOB’S BURGERS', 'THIS IS US', 'SPORTS', 'DANCING WITH THE STARS', 'HOW I MET YOUR MOTHER', 'GREY’S ANATOMY ', 'OUR FLAG MEANS DEATH', 'CHICAGO SERIES', 'FRIENDS', 'HANNIBAL', 'STRANGER THINGS', 'NOVA', 'CHICAGO FIRE', 'THE BOYS', 'THE OFFICE', "SCHITT'S CREEK", 'STAR TREK ', 'THE OFFICE', 'MONDAY NIGHT FOOTBALL ', 'CHICAGO PD', 'BLACKLIST', 'STATION ELEVEN', 'THE FIRST LADY', 'JEOPARDY ', 'HGTV SHOWS', 'WALKING DEAD', 'YOU', 'ALL AMERICAN ', 'COSBY SHOW', 'STRANGER THINGS', 'THE L WORD', 'THE WIRE', 'FIREFLY', "I DON'T HAVE ONE", 'THE SIMPSONS', 'SUPERNATURAL', 'FRIENDS ', 'JANE THE VIRGIN ', 'DATELINE', 'STRANG

In [13]:
tv_url = "https://api.themoviedb.org/3/search/tv?api_key="+movie_api

ids = []
titles = []
genre_id = []
test = []
broken = []

for tv in shows:
    search = requests.get(tv_url + "&query="+tv).json()
    # search = requests.get(movie_url + "&query= CRITICAL ROLE").json()
    try:
        tv_id = search['results'][0]['id']
        titles.append(tv)
            
    except:
        broken.append(tv)
        print(f"'{tv}' failed again")

'CHICAGO MED, FIRE, PD' failed again
'CHICAGO SERIES' failed again
'HGTV SHOWS' failed again
'I DON'T HAVE ONE' failed again
'THE SAPRANOS' failed again
'I DON'T HAVE ONE ' failed again
'EVERYONE LOVES RAYMOND' failed again
'MILWAUKEE BREWERS PREGAME' failed again
'GRAYS ANATOMY ' failed again
'NO CURRENT FAVORITE' failed again
'GRAYS ANATOMY ' failed again
'THE CBS MORNING SHOW' failed again
'RIGHT NOW STANGERS THINGS' failed again
'AVATAR THE LAST AIR BENDER' failed again
'SCORPIANS' failed again
'I DON’T HAVE ONE ' failed again
'ROCK THE PARK, PET RESCUE' failed again
'NO SPECIFIC SHOW - I LIKE MURDER MYSTERIES, FOOD NETWORK CHAMPIONSHIP COOKING CONTESTS, ROM-COMS, ETC.  ' failed again
'DON'T HAVE ONE' failed again


In [32]:
# REPLACING TYPOS AND RESULTS THAT WERE NOT A SINGLE SHOW
survey_df["tv_show"] = survey_df["tv_show"].replace(['CHICAGO MED, FIRE, PD', 'CHICAGO SERIES'],['CHICAGO MED','CHICAGO MED'])
survey_df["tv_show"] = survey_df["tv_show"].replace(['HGTV SHOWS'],['FIXER UPPER'])
survey_df["tv_show"] = survey_df["tv_show"].replace(["I DON'T HAVE ONE","I DON’T HAVE ONE "],['NONE','NONE'])
survey_df["tv_show"] = survey_df["tv_show"].replace(['THE SAPRANOS'],['THE SOPRANOS'])
survey_df["tv_show"] = survey_df["tv_show"].replace(['EVERYONE LOVES RAYMOND'],['EVERYBODY LOVES RAYMOND'])
survey_df["tv_show"] = survey_df["tv_show"].replace(['MILWAUKEE BREWERS PREGAME'],['THURSDAY NIGHT BASEBALL'])
survey_df["tv_show"] = survey_df["tv_show"].replace(['NO CURRENT FAVORITE'],['NONE'])
survey_df["tv_show"] = survey_df["tv_show"].replace(['GRAYS ANATOMY '],["GREY'S ANATOMY"])
survey_df["tv_show"] = survey_df["tv_show"].replace(['THE CBS MORNING SHOW'],['CBS MORNING NEWS'])
survey_df["tv_show"] = survey_df["tv_show"].replace(['RIGHT NOW STANGERS THINGS'],['STRANGER THINGS'])
survey_df["tv_show"] = survey_df["tv_show"].replace(['AVATAR THE LAST AIR BENDER'],['AVATAR: THE LAST AIRBENDER'])
survey_df["tv_show"] = survey_df["tv_show"].replace(['SCORPIANS'],['SCORPION'])
survey_df["tv_show"] = survey_df["tv_show"].replace(['ROCK THE PARK, PET RESCUE'],['ROCK THE PARK'])
survey_df["tv_show"] = survey_df["tv_show"].replace(['NO SPECIFIC SHOW - I LIKE MURDER MYSTERIES, FOOD NETWORK CHAMPIONSHIP COOKING CONTESTS, ROM-COMS, ETC.  '],['THE AFTER PARTY'])
survey_df["tv_show"] = survey_df["tv_show"].replace(["DON'T HAVE ONE"],['NONE'])
shows = survey_df["tv_show"].to_list()
print(shows)

['FIREFLY', 'VINCENZO', 'TED LASSO', 'SUCCESSION', 'CRITICAL ROLE', 'GOSSIP GIRL', 'DEMON SLAYER', 'WEST WING', 'FRIENDS', 'NCIS', 'NCIS', 'THE OFFICE', 'TOP CHEF', 'OUTLANDER', 'NEW GIRL', 'TWIN PEAKS', 'STRANGER THINGS', 'NONE', 'CHICAGO MED', 'NEW GIRL', 'BATTLEBOTS', 'BREAKING BAD', 'FAMILY FEUD ', 'CHICAGO MED', 'THE OFFICE', 'RICK AND MORTY', "HELL'S KITCHEN ", 'BOB’S BURGERS', 'THIS IS US', 'SPORTS', 'DANCING WITH THE STARS', 'HOW I MET YOUR MOTHER', 'GREY’S ANATOMY ', 'OUR FLAG MEANS DEATH', 'CHICAGO MED', 'FRIENDS', 'HANNIBAL', 'STRANGER THINGS', 'NOVA', 'CHICAGO FIRE', 'THE BOYS', 'THE OFFICE', "SCHITT'S CREEK", 'STAR TREK ', 'THE OFFICE', 'MONDAY NIGHT FOOTBALL ', 'CHICAGO PD', 'BLACKLIST', 'STATION ELEVEN', 'THE FIRST LADY', 'JEOPARDY ', 'FIXER UPPER', 'WALKING DEAD', 'YOU', 'ALL AMERICAN ', 'COSBY SHOW', 'STRANGER THINGS', 'THE L WORD', 'THE WIRE', 'FIREFLY', 'NONE', 'THE SIMPSONS', 'SUPERNATURAL', 'FRIENDS ', 'JANE THE VIRGIN ', 'DATELINE', 'STRANGER THINGS', 'SEINFELD', 

In [33]:
tv_url = "https://api.themoviedb.org/3/search/tv?api_key="+movie_api

ids = []
titles = []
genre_id = []
test = []
broken = []
for tv in shows:
    search = requests.get(tv_url + "&query="+tv).json()
    # search = requests.get(movie_url + "&query= CRITICAL ROLE").json()
    try:
        tv_id = search['results'][0]['id']
        titles.append(tv)
            
    except:
        broken.append(tv)
        print(f"'{tv}' failed again")